In [1]:
pip install azure-cognitiveservices-vision-customvision

  Obtaining dependency information for azure-cognitiveservices-vision-customvision from https://files.pythonhosted.org/packages/8d/da/a060c57357d0fef46c1f4c75c708eab8f5fe59b14c0151b714131ee8acfa/azure_cognitiveservices_vision_customvision-3.1.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for azure-common~=1.1 from https://files.pythonhosted.org/packages/62/55/7f118b9c1b23ec15ca05d15a578d8207aa1706bc6f7c87218efffbbf875d/azure_common-1.1.28-py2.py3-none-any.whl.metadata
  Obtaining dependency information for azure-mgmt-core<2.0.0,>=1.2.0 from https://files.pythonhosted.org/packages/ab/2d/762b027cfd58b1b2c9b5b60d112615bd04bc33ef85dac55d2ee739641054/azure_mgmt_core-1.5.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/63.0 kB ? eta -:--:--
   ---------------------------------------- 63.0/63.0 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import Region
from msrest.authentication import ApiKeyCredentials
import os

# Azure Custom Vision 설정
TRAINING_KEY = "536b9bbd0c6240a2a689a6b740f725db"         # Azure 포털에서 확인
ENDPOINT = "https://southcentralus.api.cognitive.microsoft.com/"   # Azure 포털에서 확인
PROJECT_ID = "/subscriptions/b850d62a-25fe-4d3a-9697-ea40449528a9/resourceGroups/6b014cv20250214-1/providers/Microsoft.CognitiveServices/accounts/6b014cv20250214-1"             # Custom Vision 프로젝트 ID

# ApiKeyCredentials로 인증
credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(endpoint=ENDPOINT, credentials=credentials)

# 이미지 폴더 경로 (예시)
image_folder = r"C:\Users\kimyo\Downloads\음식 데이터\한과\약과"

# Bounding Box 데이터 (예시)
bounding_boxes = {
    "Img_143_0004.jpg": [(42, 85, 247, 209, "약과")],
    "Img_143_0006.jpg": [(38, 180, 262, 195, "약과")],
    "Img_143_0007.jpg": [(0, 240, 361, 193, "약과")],
    "Img_143_0015.jpg": [(0, 168, 720, 361, "약과")],
    "Img_143_0016.jpg": [(82, 43, 237, 348, "약과")],
    "Img_143_0018.jpg": [(76, 182, 138, 118, "약과")],
    "Img_143_0019.jpg": [(163, 2, 341, 222, "약과")],
    "Img_143_0025.jpg": [(3, 126, 627, 321, "약과")],
    "Img_143_0027.jpg": [(0, 0, 251, 233, "약과")],
    "Img_143_0028.jpg": [(0, 147, 156, 193, "약과")]
}

# width, height 값 설정 (예시)
# 실제 이미지 크기에 맞게 수정 필요
width, height = 800, 640  

# 1. 프로젝트 내 모든 태그 조회
existing_tags = trainer.get_tags(PROJECT_ID)

# 2. "약과" 태그가 이미 있는지 확인
yakgwa_tag = None
for t in existing_tags:
    if t.name == "약과":
        yakgwa_tag = t
        break

# 3. "약과" 태그가 없으면 생성
if yakgwa_tag is None:
    yakgwa_tag = trainer.create_tag(PROJECT_ID, "약과")
    print('"약과" 태그가 새로 생성되었습니다.')
else:
    print('"약과" 태그가 이미 존재하므로 재활용합니다.')

# 4. 이미지 업로드와 Bounding Box 설정
for image_name, boxes in bounding_boxes.items():
    image_path = os.path.join(image_folder, image_name)
    # 파일이 실제로 존재하는지 확인 (필수는 아니지만 에러 방지를 위해 권장)
    if not os.path.exists(image_path):
        print(f"파일을 찾을 수 없습니다: {image_path}")
        continue
    
    with open(image_path, "rb") as image_data:
        regions = []
        for x, y, w, h, tag_name in boxes:
            # 태그: 이미 찾은 yakgwa_tag 사용
            regions.append(
                Region(
                    left=x/width,
                    top=y/height,
                    width=w/width,
                    height=h/height,
                    tag_id=yakgwa_tag.id
                )
            )
        # 이미지와 함께 Bounding Box 정보를 서버에 전송
        upload_result = trainer.create_images_from_data(PROJECT_ID, image_data, regions=regions)
        
        # 업로드 결과 확인 (옵션)
        if not upload_result.is_batch_successful:
            print(f"{image_name} 업로드 중 오류가 발생했습니다.")
        else:
            print(f"{image_name} 업로드 및 Bounding Box 설정 완료!")

print("작업이 완료되었습니다.")


"약과" 태그가 이미 존재하므로 재활용합니다.
Img_143_0004.jpg 업로드 중 오류가 발생했습니다.
Img_143_0006.jpg 업로드 중 오류가 발생했습니다.
Img_143_0007.jpg 업로드 중 오류가 발생했습니다.
Img_143_0015.jpg 업로드 중 오류가 발생했습니다.
Img_143_0016.jpg 업로드 중 오류가 발생했습니다.
Img_143_0018.jpg 업로드 중 오류가 발생했습니다.
Img_143_0019.jpg 업로드 중 오류가 발생했습니다.
Img_143_0025.jpg 업로드 중 오류가 발생했습니다.
Img_143_0027.jpg 업로드 중 오류가 발생했습니다.
Img_143_0028.jpg 업로드 중 오류가 발생했습니다.
작업이 완료되었습니다.
